In [ ]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow as tf
import random

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
with open('/content/drive/MyDrive/intents.json') as file:
    kb = json.load(file)

In [ ]:
print(kb)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hello, thanks for visiting', 'Good to see you again', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'gameboy', 'patterns': ['gameboy', 'Gameboy', 'game boy', 'Game Boy', 'Game-Boy', 'gbc', 'gba'], 'responses': ['creating photo realistic images with neural networks and a gameboy camera  the game boy printer  - an in-depth technical document about the printer hardware , the communication protocol and the usual routine that games used for implementing the print feature .', 'game boy cartridge pcb photos  # custom cartridges emulating a gameboy cartridge  - emulating the functionality of a game boy cartridge with the development board stm32f4 .', 'homebrew-gameboy-cartridge  - eagle library , schematic , and board files for

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in kb['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])


print(intent)

{'tag': 'tutorials', 'patterns': ['tutorials', 'tutorial'], 'responses': ['tutorials , wiring and schematics provided .', '# programming guides , tutorials and tools to develop software for game boy using the development toolchains described in the software development chapter .', 'dmgreport  - game programming tutorials in assembly .', 'simplified gbdk examples  gbdk programming video tutorials  - a series of video tutorials introducing beginners to programming with gbdk .', '# websites devrs.com/gb  - old home of the scene : examples , sources , complete documentation , guides , tutorials and various tools .', '[ endif ] projects recently updated most likes newest project lists discover contests courses stack more courses tutorials events hackerspaces profiles hackaday.com tindie marketplace sign up log in close 0 % 0 % nekocart - cpld gameboy cartridge open-source gameboy flash cartridge using an xilinx cpld as mbc wenting zhang following follow project liked like project become a h

In [ ]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = numpy.array(training)
output = numpy.array(output)

In [ ]:
tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)

Training Step: 5999  | total loss: 0.49600 | time: 0.024s
| Adam | epoch: 1000 | loss: 0.49600 - acc: 0.9713 -- iter: 40/42
Training Step: 6000  | total loss: 0.44971 | time: 0.028s
| Adam | epoch: 1000 | loss: 0.44971 - acc: 0.9742 -- iter: 42/42
--


In [ ]:
model.save("/content/drive/MyDrive/model.tflearn")

In [ ]:
import pickle 

with open("/content/drive/MyDrive/data.pickle", "wb") as f:
    pickle.dump((words, labels, training, output), f)